# 3. Tratamento de Valores Ausentes - Ames Housing Dataset

Este notebook foca no tratamento de valores ausentes, incluindo:
1. Análise detalhada de missing values
2. Estratégias de imputação
3. Validação das imputações

In [ ]:
# Importando bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.impute import SimpleImputer

# Configurações de visualização
plt.style.use('seaborn')
%matplotlib inline

In [ ]:
# Carregando os dados
df = pd.read_csv('../data/ames_housing.csv')

# Análise inicial de valores ausentes
missing_values = df.isnull().sum()
missing_percentages = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Valores Ausentes': missing_values,
    'Porcentagem (%)': missing_percentages
})

# Exibindo apenas colunas com valores ausentes
print("Colunas com valores ausentes:")
missing_df[missing_df['Valores Ausentes'] > 0].sort_values('Valores Ausentes', ascending=False)

## 1. Visualização de Padrões de Valores Ausentes

In [ ]:
# Matriz de valores ausentes
plt.figure(figsize=(12, 6))
msno.matrix(df)
plt.title('Matriz de Valores Ausentes')
plt.show()

# Dendrograma de valores ausentes
plt.figure(figsize=(12, 6))
msno.dendrogram(df)
plt.title('Dendrograma de Valores Ausentes')
plt.show()

## 2. Estratégias de Imputação

In [ ]:
def impute_numeric(df, strategy='median'):
    """Imputa valores ausentes em colunas numéricas."""
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    imputer = SimpleImputer(strategy=strategy)
    df_numeric = pd.DataFrame(imputer.fit_transform(df[numeric_cols]), columns=numeric_cols)
    return df_numeric

def impute_categorical(df):
    """Imputa valores ausentes em colunas categóricas usando o modo."""
    categorical_cols = df.select_dtypes(include=['object']).columns
    imputer = SimpleImputer(strategy='most_frequent')
    df_categorical = pd.DataFrame(imputer.fit_transform(df[categorical_cols]), columns=categorical_cols)
    return df_categorical

# Criando uma cópia do dataframe original
df_imputed = df.copy()

# Imputando valores numéricos
df_numeric_imputed = impute_numeric(df)
for col in df_numeric_imputed.columns:
    df_imputed[col] = df_numeric_imputed[col]

# Imputando valores categóricos
df_categorical_imputed = impute_categorical(df)
for col in df_categorical_imputed.columns:
    df_imputed[col] = df_categorical_imputed[col]

## 3. Validação das Imputações

In [ ]:
def compare_distributions(df_original, df_imputed, column):
    """Compara distribuições antes e depois da imputação."""
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    sns.histplot(df_original[column].dropna(), kde=True)
    plt.title(f'Distribuição Original - {column}')
    
    plt.subplot(1, 2, 2)
    sns.histplot(df_imputed[column], kde=True)
    plt.title(f'Distribuição após Imputação - {column}')
    
    plt.tight_layout()
    plt.show()

# Selecionando algumas colunas numéricas com valores ausentes para validação
numeric_cols_with_missing = df.select_dtypes(include=[np.number]).columns[
    df.select_dtypes(include=[np.number]).isnull().any()
]

for col in numeric_cols_with_missing[:3]:  # Validando as 3 primeiras colunas
    compare_distributions(df, df_imputed, col)

In [ ]:
# Salvando o dataset tratado
df_imputed.to_csv('../data/ames_housing_treated.csv', index=False)
print("Dataset tratado salvo com sucesso!")